In [4]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd

In [5]:
df = pd.read_excel('Datasets_Ready.xlsx')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4510 entries, 0 to 4509
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      4510 non-null   int64  
 1   Text            4510 non-null   object 
 2   emoji           4510 non-null   object 
 3   Tokenized       4510 non-null   object 
 4   final_text      4494 non-null   object 
 5   text_emoji      4510 non-null   object 
 6   Pos_Word        4510 non-null   int64  
 7   Neg_Word        4510 non-null   int64  
 8   Total_Word      4510 non-null   int64  
 9   Pos_Ratio       4510 non-null   float64
 10  Neg_Ratio       4510 non-null   float64
 11  Sentimen_Text   4510 non-null   object 
 12  Tokenize_Emoji  4510 non-null   object 
 13  Pos_Emoji       4510 non-null   int64  
 14  Neg_Emoji       4510 non-null   int64  
 15  Sentimen_Emoji  4510 non-null   object 
 16  Sarcasm         4510 non-null   object 
dtypes: float64(2), int64(6), object(9

,Unnamed: 0,Text,emoji,Tokenized,final_text,text_emoji,Pos_Word,Neg_Word,Total_Word,Pos_Ratio,Neg_Ratio,Sentimen_Text,Tokenize_Emoji,Pos_Emoji,Neg_Emoji,Sentimen_Emoji,Sarcasm
0,0,kawan2 sunda bersatu jemput arteria dahlan,😁,"['kawan', 'sunda', 'bersatu', 'jemput', 'arter...",kawan sunda bersatu jemput arteria dahlan,kawan sunda bersatu jemput arteria dahlan 😁,2,0,6,0.333333,0.000000,Positif,"['kawan', 'sunda', 'bersatu', 'jemput', 'arter...",1,0,Positif,Negatif
1,2,arteria dahlan disidang adat sunda daerah menu...,😁,"['arteria', 'dahlan', 'sidang', 'adat', 'sunda...",arteria dahlan sidang adat sunda daerah menunt...,arteria dahlan sidang adat sunda daerah menunt...,5,4,9,0.555556,0.444444,Positif,"['arteria', 'dahlan', 'sidang', 'adat', 'sunda...",1,0,Positif,Negatif
2,4,ga ditafsirkan membanding bandingkan edy arter...,😁,"['ga', 'tafsir', 'membanding', 'membandingkan'...",ga tafsir membanding membandingkan edy arteria...,ga tafsir membanding membandingkan edy arteria...,3,2,9,0.333333,0.222222,Positif,"['ga', 'tafsir', 'membanding', 'membandingkan'...",1,0,Positif,Negatif
3,5,nantik ngak divhumas polri nya bilang palsu td...,😁,"['nanti', 'tidak', 'divisi humas', 'polri', '....",divisi humas polri bilang palsu mengeluarkan a...,divisi humas polri bilang palsu mengeluarkan a...,4,8,16,0.250000,0.500000,Negatif,"['divisi', 'humas', 'polri', 'bilang', 'palsu'...",1,0,Positif,Positif
4,6,urut dada dech kuasa hukum tdk mengerti hukum ...,😁,"['urut', 'dada', 'deh', 'kuasa', 'hukum', 'tid...",urut dada deh kuasa hukum mengerti hukum arter...,urut dada deh kuasa hukum mengerti hukum arter...,14,6,28,0.500000,0.214286,Positif,"['urut', 'dada', 'deh', 'kuasa', 'hukum', 'men...",1,0,Positif,Negatif


In [6]:
class GloVe:
    def __init__(self, corpus, embedding_size=100, window_size=5, min_count=5):
        self.corpus = corpus
        self.embedding_size = embedding_size
        self.window_size = window_size
        self.min_count = min_count
        self.word2id = {}
        self.id2word = {}
        self.word2count = {}
        self.vocab_size = 0
        self.co_matrix = None
        self.embedding = None
        self.bias = None

    def build_vocab(self):
        for sentence in self.corpus:
            for word in sentence.split():
                if word not in self.word2id:
                    self.word2id[word] = self.vocab_size
                    self.id2word[self.vocab_size] = word
                    self.vocab_size += 1
                self.word2count[word] = self.word2count.get(word, 0) + 1

    def build_co_matrix(self):
        self.co_matrix = np.zeros((self.vocab_size, self.vocab_size))
        for sentence in self.corpus:
            sentence = sentence.split()
            for i, center_word in enumerate(sentence):
                for j in range(max(0, i - self.window_size), i):
                    context_word = sentence[j]
                    self.co_matrix[self.word2id[center_word]][self.word2id[context_word]] += 1
                    self.co_matrix[self.word2id[context_word]][self.word2id[center_word]] += 1

    def train(self, num_epochs=10, learning_rate=0.01):
        self.embedding = (np.random.rand(self.vocab_size, self.embedding_size) - 0.5) / self.embedding_size
        self.bias = np.zeros(self.vocab_size)

        for epoch in range(num_epochs):
            loss = 0
            for i in range(self.vocab_size):
                for j in range(self.vocab_size):
                    if self.co_matrix[i][j] != 0:
                        diff = (self.embedding[i] @ self.embedding[j]) + self.bias[i] + self.bias[j] - np.log(self.co_matrix[i][j])
                        loss += diff ** 2

                        grad_emb_i = diff * self.embedding[j]
                        grad_emb_j = diff * self.embedding[i]
                        grad_bias_i = diff
                        grad_bias_j = diff

                        self.embedding[i] -= learning_rate * grad_emb_i
                        self.embedding[j] -= learning_rate * grad_emb_j
                        self.bias[i] -= learning_rate * grad_bias_i
                        self.bias[j] -= learning_rate * grad_bias_j

            print('Epoch {}/{} - loss {:.4f}'.format(epoch + 1, num_epochs, loss))

    def get_embeddings(self):
        embeddings = {}
        for word, word_id in self.word2id.items():
            embeddings[word] = self.embedding[word_id]
        return embeddings

In [7]:
# Membangun model GloVe
glove_model = GloVe(df['text_emoji'])

# Membangun vocabulary
glove_model.build_vocab()

# Membangun co-occurrence matrix
glove_model.build_co_matrix()

# Melatih model GloVe
glove_model.train()

Epoch 1/10 - loss 50648.6527
Epoch 2/10 - loss 46808.3723
Epoch 3/10 - loss 45917.4182
Epoch 4/10 - loss 44638.6162
Epoch 5/10 - loss 41305.0100
Epoch 6/10 - loss 38683.9775
Epoch 7/10 - loss 37082.9298
Epoch 8/10 - loss 35149.1744
Epoch 9/10 - loss 33182.8033
Epoch 10/10 - loss 31489.1077


In [9]:
# Mendapatkan semua embedding
embeddings = glove_model.get_embeddings()

# Menyimpan embedding ke file teks
output_file = "embeddings.txt"
with open(output_file, "w", encoding="utf-8") as file:
    for word, embedding in embeddings.items():
        embedding_line = "{} {}\n".format(word, " ".join(str(value) for value in embedding))
        file.write(embedding_line)

print("Embedding telah disimpan ke file: {}".format(output_file))

Embedding telah disimpan ke file: embeddings.txt
